## Import Modules

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import swat
import os 
import sys

## Read in the data

In [ ]:
nh = pd.read_csv('COVID_19_NURSING_HOME_DATASET3F5_v2.csv')

## Clean the data
### Begin be removing columns that we won't be using during our analysis
Begin by dropping all the columns that start with z

In [ ]:
# select columns that do not have 'z' as first character to keep in dataframe
keep_cols=[c for c in nh.columns if c.lower()[:1] != 'z']

In [ ]:
# create new dataframe 'nh2' that keeps only the columns we captured above
nh2 = nh[keep_cols]

In [ ]:
# look at the dimensions of the df, column names, number of null entries, and data types of each column
nh2.info()

In [ ]:
# Quick look at the target varaible
sns.histplot(nh2['Residents_Weekly_Confirmed_COVID'])

In [ ]:
# drop empty column
nh3 = nh2.drop('Total_Nursing_Minutes', axis=1) 

In [ ]:
# making a copy of df is easy
nh4 = nh3.copy()

In [ ]:
# Convert column 'Week_Ending' to date format using pandas 'to_datetime' function
nh4.loc[:,"Week_Ending"]=pd.to_datetime(nh3['Week_Ending'])

 Check to see at which date the count data switches from observed values (with decimal digits .000000) to projected values (with more significant decimal digits)

In [ ]:
# Capture the last 20 rows of the date and COVID count columns
nh4[['Week_Ending', 'Residents_Total_Confirmed_COVID_', 'Residents_Weekly_Confirmed_COVID']].tail(20)

After November 1st, projected numbers (not observed values) were placed in the table. We want to get rid of these values so we are only working with real observed data.

In [ ]:
start_date = '2020-11-01'

In [ ]:
# create new df 'nh5' that filters 'nh4' to include only rows for which 'Week Ending' is <= Nov 1st
nh5 = nh4.loc[nh4['Week_Ending'] <= start_date]

In [ ]:
sns.histplot(nh5['Residents_Weekly_Confirmed_COVID'])

In [ ]:
# create boolean series that indicates whether each nh5 column has datatype object ('== object')
categorical_features = nh5.dtypes == object;

In [ ]:
# subset nh5 column names for which above code returned True (where dtype is object)
categorical_cols = nh5.columns[categorical_features].to_list() # convert to list, otherwise type would be 'index'

In [ ]:
# get counts of unique categories in each of the categorical columns by applying pandas function Series.nunique
nh5[categorical_cols].apply(pd.Series.nunique)

Check to see if the same information is provided by columns 'State' and 'Provider_State'

In [ ]:
# look at unique values of column 'State', sorted alphabetically
nh5['State'].sort_values().unique()

In [ ]:
# appears that State and Provider_State are supplying the same information
nh5['Provider_State'].sort_values().unique()

In [ ]:
# confirm that state and provider state are supplying the same information
(nh5['Provider_State'] == nh5['State']).sum()

In [ ]:
# are cms_region_location and cms_region providing equivalent information?
nh5[['CMS_REGION_LOCATION', 'CMS_REGION']].head()

In [ ]:
# get all unique combinations of the two variables
location_region = nh5['CMS_REGION_LOCATION'] + ' - ' + nh5['CMS_REGION']
pd.DataFrame(location_region.unique())

Categorical Columns to Drop: CMS_REGION, Summary_Category, Service_Category, Provider_ID, Provider_State

In [ ]:
nh6 = nh5.drop(['CMS_REGION', 'Summary_Category', 'Service_Category', 'Provider_ID', 'Provider_State'], axis=1)

In [ ]:
# select numeric cols using pandas .select_dtypes function
numeric_cols = nh6.select_dtypes(include = ['int64', 'float64'])
numeric_cols.head()

In [ ]:
# summarize continous variarbles
numeric_cols.describe()

In [ ]:
# scatterplot resident cases vs. staff cases
plt.figure(figsize = (10, 6))
plt.scatter(nh6['Residents_Weekly_Confirmed_COVID'], nh6['Staff_Weekly_Confirmed_COVID_19'])
plt.xlabel('Residents Weekly Confirmed COVID')
plt.ylabel('Staff Weekly Confirmed COVID')

In [ ]:
# take a look at the target variable over time for all states in the data
nh_ts = nh6.pivot(index='Week_Ending', columns='State', values='Residents_Weekly_Confirmed_COVID')
nh_ts.plot(figsize = (20, 12))

In [ ]:
# plot pairwise correlations as a heatmap
plt.figure(figsize=(17,10))
sns.heatmap(nh6.corr(), annot=True, fmt='.2f')

Perfect positive correlation between Episode_or_Stay_Count and Distinct_Beneficiaries, which means we should drop one of them. Perfect negative correlation between Percent_Female_Beneficiaries and Percent_Male_Beneficiaries, so one should be dropped. 

## Feature Engineering
### Create target variable
The original idea was to predict for number of deaths / cases. I want to investigate if this would really be a good predictor since there is a lag between positive case confirmation and death of the patient. I will make some plots to explore this possible relationship

In [ ]:
# aggregate state data together to generate sum of total national weekly resident cases and deaths 
weekly_cases_deaths = nh6.groupby('Week_Ending')[['Residents_Weekly_Confirmed_COVID', 'Residents_Weekly_COVID_19_Deaths']].sum()

In [ ]:
# plot total weekly cases and weekly deaths together in same figure
fig, ax1 = plt.subplots() #create figure object and axis object
color = 'tab:orange'
ax1.plot(weekly_cases_deaths.index, weekly_cases_deaths['Residents_Weekly_Confirmed_COVID'], color=color) #plot (x,y,color) 
ax1.tick_params(axis='y', labelcolor=color) #tick_params allows for the change of ticks, tick labels, and gridlines
ax1.set_xlabel('Week')
ax1.set_ylabel('Weekly Confirmed Covid Cases', color=color)

ax2 = ax1.twinx() #new axis object with same x as ax1 (Week)
color = 'tab:blue'
ax2.plot(weekly_cases_deaths.index, weekly_cases_deaths['Residents_Weekly_COVID_19_Deaths'], color=color)
ax2.set_ylabel('Weekly Confirmed Covid Deaths', color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_title('Weekly Covid Cases vs Weekly Covid Deaths') #add a title

fig.tight_layout() #prevantative in case the subplot does not fit into the figure area

In [ ]:
# examine beginning of time series where anomolously high counts occur
weekly_cases_deaths.head()

In [ ]:
# plot time series of week ending vs. confirmed cases for NC, UT, TX, and FL
fig, ax = plt.subplots()
ax.plot(nh6.loc[nh6['State'] == 'NC']['Week_Ending'], nh6.loc[nh6['State'] == 'NC']['Residents_Weekly_Confirmed_COVID'], label='NC')
ax.plot(nh6.loc[nh6['State'] == 'UT']['Week_Ending'], nh6.loc[nh6['State'] == 'UT']['Residents_Weekly_Confirmed_COVID'], label='UT')
ax.plot(nh6.loc[nh6['State'] == 'TX']['Week_Ending'], nh6.loc[nh6['State'] == 'TX']['Residents_Weekly_Confirmed_COVID'], label='TX')
ax.plot(nh6.loc[nh6['State'] == 'FL']['Week_Ending'], nh6.loc[nh6['State'] == 'FL']['Residents_Weekly_Confirmed_COVID'], label='FL')
ax.set_ylabel('Residents_Weekly_Confirmed_COVID')
ax.set_xlabel('Week_Ending')
ax.legend()

In [ ]:
# examine data for NC
nh6.loc[nh6['State'] == 'NC'][['State', 'Week_Ending', 'Residents_Weekly_Confirmed_COVID']].head(10)

First four weeks' data seem erroneously high (see graphic of all states). We will ommit this data.

In [ ]:
start_dt = '2020-05-31'

In [ ]:
# filter to include only weeks greater than or equal to start date
nh7 = nh6.loc[nh6['Week_Ending'] >= start_dt ]

In [ ]:
# check results -- examine NC case counts again
nh7.loc[nh7['State'] == 'NC'][['State', 'Week_Ending', 'Residents_Weekly_Confirmed_COVID']].head(10)

In [ ]:
# create line plot for all states showing weekly confirmed covid cases 
nh_ts = nh7.pivot(index='Week_Ending', columns='State', values='Residents_Weekly_Confirmed_COVID')
nh_ts.plot(figsize = (20, 12))

### Make new dataframe based off of time aggregated values
Get a 4 week moving average of all numeric variables

In [ ]:
# we will use the (# rows, # cols) later for a sanity check
nh7.shape

In [ ]:
nh7.Residents_Weekly_Confirmed_COVID[0:3]

Explanation of what is going on in the following cell:
1. Initialize ma_df data frame, and indentify unique states in State column. 
2. For every column in the nh7 data frame, if the column name starts with 'Resi', 'Staf', or 'Numb' then assign j the column number and initialize an empty list called ma_data. 
3. For every unique state in the state column, create a df with rows only for that state and reset the index of df since the rows keep their indices from nh7. 
4. For every row in the df data frame, assign null values to the ma_data list if its one of the first three rows; otherwise, apply a moving average of the current row and the previous three. The moving average is appended to the ma_data list and rounded to two decimals. 
5. Add the created list as a column to the ma_df data frame.

In [ ]:
# create a data frame of moving averages made from the numeric columns in nh7 df

#1
ma_df = pd.DataFrame()
unique_states = nh7['State'].unique()

#2
for column in nh7.columns.to_list():
    if column[0:4] in ['Resi', 'Staf','Numb']: 
        j = np.argmax(nh7.columns == column)
        ma_data = [] 
        #3
        for state in unique_states: 
            df = nh7.loc[nh7['State'] == state].copy() 
            df.reset_index(inplace=True, drop=True)
            #4
            for i in range(df.shape[0]): 
                if i in [0, 1, 2]: 
                    ma_data.append(None) 
                else:
                    ma_data.append(np.round((df.iloc[i,j]+df.iloc[i-1,j]+df.iloc[i-2,j]+df.iloc[i-3,j])/4,2))
        #5
        ma_df['MA_'+column] = ma_data

In [ ]:
# look at the dimensions of the ma_df dataframe as a sanity check that the loop worked
ma_df.shape

In [ ]:
# examine first 5 rows of the ma_df dataframe
ma_df.head()

### Create Target Variable 
Target variable ma_infection_beds is a moving average of the current row and the next row (looking into the future)
1. create empty list variable ma_infection_beds
2. for every unique state in the State column, create a dataframe df and reset the index
3. for every row in df, if it is the last row or second-to-last row, then give it a null value. Otherwise, give it a two day moving average.

In [ ]:
#1
ma_infection_beds = []

#2
for state in nh7['State'].unique():
    df = nh7.loc[nh7['State'] == state].copy()
    df.reset_index(inplace=True, drop=True)
    #3
    for i in range(df.shape[0]):
        if i in [df.shape[0]-1, df.shape[0]-2]:
            ma_infection_beds.append(None)
        else:
            ma_infection_beds.append((df.iloc[i+2,2]/df.iloc[i+2,6] * 100 + df.iloc[i+1,2]/df.iloc[i+1,6] * 100)/2)

In [ ]:
# check length of ma_infection_beds to make sure we got the expected number of values
len(ma_infection_beds)

In [ ]:
# add the moving average to the moving average data frame ma_df
ma_df['MA_Infection_Beds'] = ma_infection_beds

In [ ]:
nh7.tail()

### Combined DataFrames
We will be combining the MA (moving average) dataframe with columns from the nh7 dataframe to make our final data frame

In [ ]:
# make a list of columns you want to keep form nh7
keep_cols = []
for name in nh7.columns:
    if name[0:4] not in ['Week', 'Resi', 'Staf', 'Numb']:
        keep_cols.append(name)

In [ ]:
# check if indices line up
print('ma_df max indice: ', max(ma_df.index))
print('nh7 max indice: ', max(nh7.index))

In [ ]:
# reset the index to make sure the concatenation of the two df works as planned
nh7.reset_index(inplace=True, drop=True)

In [ ]:
max(nh7.index)

In [ ]:
nh8 = pd.concat([ma_df, nh7[keep_cols]], axis=1)

In [ ]:
nh8.head()

## Increase Normality of Feature Distributions
As you can see in the charts below, many of our independent variable are left or right skewed. PCA assumes that the features follow a Gaussian distribution, so to use this method we will need to apply some sort of transformation to the data. My first choice is a log transformation, but due to many of the numeric variables have 0's, we will use the cubed root transformation. Note above that went ahead and created our target and saved it to a separate variable. We will replace the transformed target variable later with the original.

In [ ]:
nh8.hist(figsize=(20,15))

Cube root transformation is suggested with zero-inflated data b/c logistic is not feasible

Features to give a cube root transformation to:
- Staff_Weekly_Confirmed_COVID
- Residents_Total_Confirmed_COVID_
- Residents_Total_COVID_19_Deaths
- Number_of_All_Beds
- Staff_Total_Confirmed_Covid_19
- Staff_Total_COVID_19_Deaths
- Distinct_Beneficiaries
- Episode_or_Stay_Count
- Total_Charge_Amount
- Percent_White_Beneficiaries 
- Percent_Black_Beneficiaries
- Percent_Asian_Pacific_Islander_Beneficiaries
- Percent_Hispanic_Beneficiaries (consider dropping that one outlier
- Percent_American_Indian_or_Alaska_Native
- Average_HCC_Score
    
I would suggest for an easy dimension reduction to have white and not-white instead of all the individual races. 

In [ ]:
# loop through numeric columns and perform cube root transformation
# rename new columns by adding prefix 'cbrt_'
numerics = ['float64', 'int64']
for c in nh8.columns:
    if nh8[c].dtype in numerics:
        new_col = 'cbrt_' + c
        nh8[new_col] = np.cbrt(nh8[c])
        nh8.drop([c], inplace=True, axis=1)

In [ ]:
nh8.columns

In [ ]:
nh8.hist(figsize=(23,16))

## Outlier Removal and Comparison
We will examine plots of the data to check for outlies and deal with them as we see necessary. Remember that while some outliers will negatively affect the model, others add explainations to unexplored space in the model that could be valuable for prediction. Because we are looking at predicting by state, there aren't many data points so determining outliers would be challenging. Removing datapoints could also be detrimental to the model since there isn't much to begin with.

In [ ]:
# look at how many data points per state
for state in ['UT', 'NC', 'TX']:
    print(state, 'datapoints: ', (nh8['State']=="UT").sum())

## Final Cleaning
 - Get rid of the myweek variable
 - untransform (or use the original column) for MA_Infection_Beds because it will make it easier so we don't have to untransform the predictions
 - dummy variables for the categorical variables, but don't do leave one out for PCA because the rbgf doesn't require it, and neither does looking at random forest imporance

In [ ]:
# replace the cuberooted values with the original move value values for infection beds
nh8['MA_Infection_Beds'] = ma_df['MA_Infection_Beds']

In [ ]:
# drop unnecessary columns 
nh8.drop(['cbrt_MA_Infection_Beds', 'cbrt_myweek', 'cbrt_Distinct_Beneficiaries', 'cbrt_Percent_Male_Beneficiaries'], axis=1, inplace=True)

In [ ]:
sns.regplot(x=nh8['cbrt_MA_Residents_Weekly_Confirmed_COVID'], y=nh8['MA_Infection_Beds'])

In [ ]:
# confirm values were dropped
nh8.info()

In [ ]:
# separate data into categorical and numeric
cat_features = nh8.dtypes==object
num_features = nh8.dtypes!=object

In [ ]:
# create separate data frames
categorical_cols = nh8.columns[cat_features].to_list()
numeric_cols = nh8.columns[num_features].to_list()

In [ ]:
# use Pandas built in get_dummies for one-hot encoding
dummies = pd.get_dummies(nh8[categorical_cols], )

In [ ]:
dummies.head()

In [ ]:
# combine the dummy-variable df with the numeric df
nh9 = pd.concat([nh8[numeric_cols],dummies], axis=1)

Explore missing values, check to see how many and whether appear missing at random

In [ ]:
# Summarize number of NA values in each column
nh9.info()

In [ ]:
#Drop NA Values
nh10 = nh9.dropna()

In [ ]:
nh10.shape

In [ ]:
# X is a df of the independent variables
X = nh10.drop('MA_Infection_Beds', axis=1)

In [ ]:
# y is  a Pandas series of the target variable
y = nh10['MA_Infection_Beds']

In [ ]:
# quick summary statistics of the target
y.describe()

In [ ]:
sns.histplot(y)
print(sum(y==0))

## Test and Train Split

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
# split the data into train and test sets where 70% of the data is for training 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=76)

In [ ]:
X_train.shape

In [ ]:
X_train.describe()

In [ ]:
y_train.describe()

## Variable Selection

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

### Lasso Regression - Standardization

Before running lasso regression, standardize design matrix (this is a model assumption).
Use only the training set to fit scaler to prevent any leakage of information from test set

In [ ]:
# initiate model scaler
scaler = StandardScaler()

In [ ]:
# fit scaler model to X_train
scaler.fit(X_train)

In [ ]:
# transform X_train using scaler model
X_train_z = scaler.transform(X_train)

In [ ]:
X_test_z = scaler.transform(X_test)

In [ ]:
# note that the scaled data is now a np array and no longer a pd data frame
type(X_test_z)

In [ ]:
df = pd.DataFrame(X_train_z)

The dummy variables do not need to be standardized so we are going to replace their standardized values with their original one-hot encoded values.

In [ ]:
# replace transformed dummy variables with original values
# get column indeces of dummy variables from nh10
col1 = nh10.columns[nh10.dtypes == 'uint8'][0]
print(col1)

# -1 because argmax returns the index starting at 1 instead of 0
first_dummy_var = np.argmax(nh10.columns == col1) - 1
print(np.argmax(nh10.columns == col1))
print(first_dummy_var)

In [ ]:
# get the untransformed dummy variables from the original X_train and X_test
X_train_dummies = X_train.iloc[:,first_dummy_var:]
X_test_dummies = X_test.iloc[:,first_dummy_var:]

In [ ]:
# make two df's to join
std_values = pd.DataFrame(X_train_z, columns=X.columns).iloc[:,0:first_dummy_var]
unstd_dummies = X_train.iloc[:,first_dummy_var:]

# same for test data
std_values_test = pd.DataFrame(X_test_z, columns=X.columns).iloc[:,0:first_dummy_var]
unstd_dummies_test = X_test.iloc[:,first_dummy_var:]

In [ ]:
# recall that pulling from a subset df means we need ot reset the index before combining tables
unstd_dummies.reset_index(inplace=True, drop=True)
unstd_dummies_test.reset_index(inplace=True, drop=True)

In [ ]:
# combine the two df's into one
X_train_std = pd.concat([std_values, unstd_dummies], axis=1)
X_test_std = pd.concat([std_values_test, unstd_dummies_test], axis=1)

In [ ]:
# confirm shape is as expected
X_train_std.shape

In [ ]:
X_test_std.shape

### Lasso Regression - Model

In [ ]:
# instantiate model

# alpha = lambda for lasso regression 
# test 200 different alpha values to find optimal
# cv = k-fold cross-validation, k=10
nh_lassocv = LassoCV(alphas=None, n_alphas=200, cv=10, max_iter=1000000, random_state=76, fit_intercept=False, tol=0.0001)

In [ ]:
# fit cross validation model to find best alpha (lambda)
nh_lassocv.fit(X_train_std, y_train)

In [ ]:
# the lapha is very small indicating that our model will be very close to a regular linear model
nh_lassocv.alpha_

In [ ]:
# fit our lasso regression model
nh_lasso = Lasso(alpha=nh_lassocv.alpha_)
nh_lasso.fit(X_train_std, y_train)

In [ ]:
coef_df = pd.DataFrame({'Coefficients': nh_lasso.coef_}, index=X_train.columns)

In [ ]:
coef_df.sort_values(by=['Coefficients'], ascending=False)

One of the big advantages to using Lasso Regression for model selection is that it's regularization penalty pushes many of the regression coefficients to zero. This can help us see what variables we should consider dropping for future models. As you can see, many of the region locations and states have coefficients of 0 meaning they do not contribute to the model.=

In [ ]:
# create a data frame of coefficients that are between -0.01 and 0.01
close_to_zero = (coef_df['Coefficients'] > -0.01) & (coef_df['Coefficients'] < 0.01)

In [ ]:
coef_df.loc[close_to_zero]

In [ ]:
lasso_mse = mean_squared_error(nh_lasso.predict(X_test_std), y_test)
print(mean_squared_error(nh_lasso.predict(X_test_std), y_test))
# note that because of the convergence issue I would not recommend using this model.
# possible remedies to the convergence issue incude changering the tolerance, drastically increase iterations, or decreasing the number of variables

## Modeling

### Random Forest 

In [ ]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=76)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
importance = pd.DataFrame({'Importance': rf.feature_importances_*100}, index=X_train.columns)
importance = importance.iloc[rf.feature_importances_*100 > 1, :]
#importance = importance.sort_values('Importance', axis=1, ascending=False)

Variable importance in random forests is not the same as regression coefficients. Variable importance is relative to all the other variables, and tells you how much information is gained if that variable is frequently chosen as the feature to split on across multiple trees in the forest.

In [ ]:
importance.plot(kind='barh', color='r')
plt.xlabel('Variable Importance')

In [ ]:
print(mean_squared_error(rf.predict(X_test), y_test))
rf_mse = mean_squared_error(rf.predict(X_test), y_test)

### PCA - LM

In [ ]:
n_components = np.arange(1,25)
pca_score = []

for n in n_components:
    pca = PCA(n_components=n)
    pca.fit(X_train_std)
    X_pca = pca.transform(X_train_std)
    pca_lm = LinearRegression()
    score = cross_val_score(pca_lm, X_pca, y_train, scoring='neg_mean_squared_error').mean()
    pca_score.append(score)

In [ ]:
n = n_components[np.argmax(pca_score)]
print(n)

In [ ]:
pca = PCA(n_components=n)
pca.fit(X_train_std)
X_pca = pca.transform(X_train_std)
pca_lm = LinearRegression()
pca_lm.fit(X_pca, y_train)

In [ ]:
X_test_pca = pca.transform(X_test_std)

In [ ]:
pca_lm_mse = mean_squared_error(y_test, pca_lm.predict(X_test_pca))
print(mean_squared_error(y_test, pca_lm.predict(X_test_pca)))

### PCA - RF

In [ ]:
n_components = np.arange(1,25)
pca_score = []

for n in n_components:
    pca = PCA(n_components=n)
    pca.fit(X_train_std)
    X_pca = pca.transform(X_train_std)
    pca_rf = RandomForestRegressor(random_state=76)
    score = cross_val_score(pca_rf, X_pca, y_train, scoring='neg_mean_squared_error').mean()
    pca_score.append(score)

In [ ]:
n = n_components[np.argmax(pca_score)]
print(n)

In [ ]:
pca = PCA(n_components=n)
pca.fit(X_train_std)
X_pca = pca.transform(X_train_std)
pca_rf = RandomForestRegressor(random_state=76)
pca_rf.fit(X_pca, y_train)

In [ ]:
X_test_pca = pca.transform(X_test_std)

In [ ]:
pca_rf_mse = mean_squared_error(y_test, pca_rf.predict(X_test_pca))
print(mean_squared_error(y_test, pca_rf.predict(X_test_pca)))

### XG Boost

XGBoost is a specific Gradeint Boosting Machine, with optimization differences compared to traditional GBM's. XGBoost became popular after multiple Kaggle competitions were won using it.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
# instatiate model for cross validation and hyperparameter selection
boost_cv = XGBRegressor(random_state=76)

In [ ]:
# create list of possibly hyperparameter values
learning_rates = [0.001, 0.01, 0.1, 0.2, 0.3]

In [ ]:
# make a dictionar of the learning rates you want to try
grid = dict(learning_rate = learning_rates)

In [ ]:
# do a grid search using 5 kfold cross validation measuring by negative mean squared error
grid_search = GridSearchCV(boost_cv, grid, scoring="neg_mean_squared_error", n_jobs=-1, cv=5)

In [ ]:
# fit grid result ot data
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
# look at results
grid_result.cv_results_['mean_test_score']

In [ ]:
# find our learning rate based on nmse
learning_rates[np.argmax(grid_result.cv_results_['mean_test_score'])-1]

In [ ]:
boost = XGBRegressor(random_state=76, learning_rate=0.2)

In [ ]:
boost.fit(X_train, y_train)

In [ ]:
mse_lr2 = [mean_squared_error(boost.predict(X_test), y_test), 0.2]

In [ ]:
gbx_mse = mean_squared_error(boost.predict(X_test), y_test)
print(gbx_mse)

## Choose champion model based on MSE

In [ ]:
print("Lasso: " + str(round(lasso_mse, 3)))
print("Random Forest: " + str(round(rf_mse, 3)))
print("PCA LM: " + str(round(pca_lm_mse, 3)))
print("PCA RF: " + str(round(pca_rf_mse, 3)))
print("XGBoost: " + str(round(gbx_mse, 3)))

## Export the XGBoost model

In [ ]:
from sasctl import Session, register_model, publish_model

In [ ]:
host="sasserver.demo.sas.com"
my_username='sasdemo'
my_password='Orion123'

In [ ]:
# Establish a CAS session - there will be a warning because 
s = Session(hostname=host, username=my_username, password=my_password, verify_ssl=False)

In [ ]:
# set model and project name
model_name = 'XGBoost Regressor NH3'
project_name = 'Nursing Home Covid-19'

# Register the model in SAS Model Manager
model = register_model(boost, model_name, project_name, input=X_train, force=True)

## Export train and test data to CAS

### Make Connection to CAS using SWAT

In [ ]:
#Connect to CAS Server
host="sasserver.demo.sas.com"
portnumber = 5570
my_username='sasdemo'
my_password='Orion123'

In [ ]:
sess=swat.CAS(host, portnumber, my_username, my_password)
#sess.close()

In [ ]:
# Verify Connection
sess.serverstatus()

In [ ]:
sess.table.caslibinfo(active="True")

In [ ]:
# view available caslibs and create NH caslib if not already done
sess.table.caslibinfo()
#sess.addcaslib(name='NH', path='/home/sasdemo/Python/SGF/', session=False, datasource={'srctype':'path'})

In [ ]:
# change active caslib to Public
sess.sessionProp.setSessOpt(caslib="NH")

In [ ]:
sess.table.caslibinfo(active="True")

In [ ]:
# create train and test pandas df
train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)

In [ ]:
train.shape

In [ ]:
# read in pandas DataFrames to CAS
sess.read_frame(train, casout={'caslib':'NH', 'name':'nh_train', 'promote':True})
sess.read_frame(test, casout={'caslib':'NH', 'name':'nh_test', 'promote':True})

In [ ]:
# view that the tables were loaded into memory
sess.table.tableinfo(caslib='NH')